In [346]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_venn import venn3
from sklearn import svm
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.preprocessing import normalize
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from sklearn.ensemble import GradientBoostingClassifier

In [316]:
# load all datasets
allRNA = pd.read_csv("data/200625_allRNA_fromRNAseq_annot_hg38.tsv", sep="\t")
circRNA = pd.read_csv("data/200625_circRNA_fromRNAseq_annot_hg19.tsv", sep="\t")
circRNA.fillna(0, inplace = True)
miRNA = pd.read_excel("data/final_all_samples_miRNA_seq.xlsx")
piRNA = pd.read_excel("data/piRNA_counts.xlsx")
TE = pd.read_csv("data/TE_counts.csv")
annot = pd.read_excel("data/sample sheet for CVUT.xlsx")

In [317]:
# modifying column names so that they only contain patient ID
# annotation file -> modifying row values
id_allRNA = [i.split("_")[0] for i in allRNA.columns[6:]]
id_circRNA = [i.split("_")[0] for i in circRNA.columns[9:]]
id_miRNA = [i.split("_")[0] for i in miRNA.columns[1:]]
id_piRNA = [i.split("_")[0] for i in piRNA.columns[1:]]
id_TE = [i.split("_")[0] for i in TE.columns[1:]]
id_annot = [i.split("_")[0] for i in annot.SAMPLE_NAME]

for i in range(len(id_allRNA)):
    allRNA.rename(columns = {allRNA.columns[i+6]: id_allRNA[i]}, inplace = True)
for i in range(len(id_circRNA)):
    circRNA.rename(columns = {circRNA.columns[i+9]: id_circRNA[i]}, inplace = True)
for i in range(len(id_miRNA)):
    miRNA.rename(columns = {miRNA.columns[i+1]: id_miRNA[i]}, inplace = True)
for i in range(len(id_piRNA)):
    piRNA.rename(columns = {piRNA.columns[i+1]: id_piRNA[i]}, inplace = True)
for i in range(len(id_TE)):
    TE.rename(columns = {TE.columns[i+1]: id_TE[i]}, inplace = True)
annot.SAMPLE_NAME = id_annot

In [344]:
print("Number of IDs in each dataset:\n")
for ds, name in zip((id_annot, id_allRNA, id_circRNA, id_miRNA, id_piRNA, id_TE),
                    ("annot", "allRNA", "circRNA", "miRNA", "piRNA", "TE")):
    print(f"{name}: {len(ds)}")

Number of IDs in each dataset:

annot: 98
allRNA: 86
circRNA: 86
miRNA: 105
piRNA: 104
TE: 112


In [342]:
print("Number of common IDs between annotation file and each dataset:\n")
for ds, name in zip((id_allRNA, id_circRNA, id_miRNA, id_piRNA, id_TE),
                    ("allRNA", "circRNA", "miRNA", "piRNA", "TE")):
    common = 0
    for i in list(annot.SAMPLE_NAME):
        if i in ds:
            common += 1
    print(f"{name}: {common}")

Number of common IDs between annotation file and each dataset:

allRNA: 75
circRNA: 75
miRNA: 85
piRNA: 85
TE: 97


In [318]:
# creating a list of all IDs that are present in all datasets
complete_id = []  # list of tuples [(id1_allRNA, id1_miRNA, id1_piRNA, id1_TE), (id2...)]
for i in id_allRNA:
    if i in id_miRNA and i in id_piRNA and i in id_circRNA and i in list(annot.SAMPLE_NAME):
        complete_id.append(i)
print("Number of IDs with complete data: ", str(len(complete_id)))

Number of IDs with complete data:  66


In [319]:
# data concatenation into final dataset
# row = id, column = feature
# last 3 rows are target values
complete_data = np.zeros((len(complete_id), allRNA.shape[0]+circRNA.shape[0]+miRNA.shape[0]+piRNA.shape[0]+TE.shape[0] + 3))
for i, p in enumerate(complete_id):
    allRNA_data = allRNA[p].to_numpy()
    circRNA_data = circRNA[p].to_numpy()
    miRNA_data = miRNA[p].to_numpy()
    piRNA_data = piRNA[p].to_numpy()
    TE_data = TE[p].to_numpy()
    annot_data = annot.loc[annot['SAMPLE_NAME'] == complete_id[i]]
    annot_data = annot_data.iloc[:, 2:5].to_numpy()[0]
    row = np.concatenate((allRNA_data, circRNA_data, miRNA_data, piRNA_data, TE_data, annot_data))
    complete_data[i, :] = row

In [320]:
# divide into features and target
X = complete_data[:, :-3]
XN = normalize(X, axis = 0)
Y_disease = complete_data[:, -3]
Y_risk = complete_data[:, -2]
Y_mutation = complete_data[:, -1]

In [294]:
# linear model with L1 regularization
# LM = Lasso(alpha = 5.0, tol = 0.005)  # proves to be useless
C_vals = [200, 500, 800, 1000]

In [295]:
LR_CVscores_disease = []
LR_CVscores_risk = []
LR_CVscores_mutation = []

for i, c in enumerate(C_vals):
    LR = LogisticRegression(penalty = "l1", C = c, tol = 0.01, solver = "saga")
    LR_CVscores_disease.append(cross_val_score(LR, XN, Y_disease, cv = 6))
    LR_CVscores_risk.append(cross_val_score(LR, XN, Y_risk, cv = 6))
    LR_CVscores_mutation.append(cross_val_score(LR, XN, Y_mutation, cv = 6))

In [304]:
print("Accuracy - LogReg:\n")
for i, c in enumerate(C_vals):
    print("C =", str(c))
    print(f"Disease: {np.mean(LR_CVscores_disease[i]):.2} +/- {np.std(LR_CVscores_disease[i]):.2}")
    print(f"Risk: {np.mean(LR_CVscores_risk[i]):.2} +/- {np.std(LR_CVscores_risk[i]):.2}")
    print(f"Mutation: {np.mean(LR_CVscores_mutation[i]):.2} +/- {np.std(LR_CVscores_mutation[i]):.2}")

Accuracy - LogReg:

C = 200
Disease: 0.89 +/- 0.034
Risk: 0.58 +/- 0.15
Mutation: 0.62 +/- 0.034
C = 500
Disease: 0.89 +/- 0.034
Risk: 0.58 +/- 0.15
Mutation: 0.62 +/- 0.034
C = 800
Disease: 0.89 +/- 0.034
Risk: 0.58 +/- 0.15
Mutation: 0.62 +/- 0.034
C = 1000
Disease: 0.89 +/- 0.034
Risk: 0.58 +/- 0.15
Mutation: 0.62 +/- 0.034


In [297]:
SVM_CVscores_disease = []
SVM_CVscores_risk = []
SVM_CVscores_mutation = []
n_select = [5, 10, 20, 100]

In [300]:
SVM = svm.SVC(kernel = "linear", C = 1.0)
for n in n_select:
    selector = RFE(SVM, n_features_to_select = n, step = 500)
    selector = selector.fit(XN, Y_disease)
    selected = np.where(selector.support_ == True)[0]
    XN_selected = XN[:, selected]
    SVM_CVscores_disease.append(cross_val_score(SVM, XN_selected, Y_disease, cv = 6))
    selector = selector.fit(XN, Y_risk)
    selected = np.where(selector.support_ == True)[0]
    XN_selected = XN[:, selected]
    SVM_CVscores_risk.append(cross_val_score(SVM, XN_selected, Y_risk, cv = 6))
    selector = selector.fit(XN, Y_mutation)
    selected = np.where(selector.support_ == True)[0]
    XN_selected = XN[:, selected]
    SVM_CVscores_mutation.append(cross_val_score(SVM, XN_selected, Y_mutation, cv = 6))

In [305]:
print("Accuracy - SVM:\n")
for i, n in enumerate(n_select):
    print("N =", str(n))
    print(f"Disease: {np.mean(SVM_CVscores_disease[i]):.2} +/- {np.std(SVM_CVscores_disease[i]):.2}")
    print(f"Risk: {np.mean(SVM_CVscores_risk[i]):.2} +/- {np.std(SVM_CVscores_risk[i]):.2}")
    print(f"Mutation: {np.mean(SVM_CVscores_mutation[i]):.2} +/- {np.std(SVM_CVscores_mutation[i]):.2}")

Accuracy - SVM:

N = 5
Disease: 0.92 +/- 0.034
Risk: 0.44 +/- 0.062
Mutation: 0.62 +/- 0.034
N = 10
Disease: 0.92 +/- 0.034
Risk: 0.52 +/- 0.043
Mutation: 0.67 +/- 0.068
N = 20
Disease: 0.92 +/- 0.034
Risk: 0.67 +/- 0.068
Mutation: 0.7 +/- 0.068
N = 100
Disease: 0.94 +/- 0.043
Risk: 0.89 +/- 0.082
Mutation: 0.82 +/- 0.074


In [312]:
XGB = GradientBoostingClassifier(max_depth = 8, learning_rate = 1.0)

In [313]:
XGB_CVscores_disease = []
XGB_CVscores_risk = []
XGB_CVscores_mutation = []

for n in n_select:
    XGB.fit(XN, Y_disease)
    idx = np.argpartition(XGB.feature_importances_, -n)[-n:]
    XN_selected = XN[:, idx]
    XGB_CVscores_disease.append(cross_val_score(XGB, XN_selected, Y_disease, cv = 6))
    XGB.fit(XN, Y_risk)
    idx = np.argpartition(XGB.feature_importances_, -n)[-n:]
    XN_selected = XN[:, idx]
    XGB_CVscores_risk.append(cross_val_score(XGB, XN_selected, Y_risk, cv = 6))
    XGB.fit(XN, Y_mutation)
    idx = np.argpartition(XGB.feature_importances_, -n)[-n:]
    XN_selected = XN[:, idx]
    XGB_CVscores_mutation.append(cross_val_score(XGB, XN_selected, Y_mutation, cv = 6))

In [314]:
print("Accuracy - XGB:\n")
for i, n in enumerate(n_select):
    print("N =", str(n))
    print(f"Disease: {np.mean(XGB_CVscores_disease[i]):.2} +/- {np.std(XGB_CVscores_disease[i]):.2}")
    print(f"Risk: {np.mean(XGB_CVscores_risk[i]):.2} +/- {np.std(XGB_CVscores_risk[i]):.2}")
    print(f"Mutation: {np.mean(XGB_CVscores_mutation[i]):.2} +/- {np.std(XGB_CVscores_mutation[i]):.2}")

Accuracy - XGB:

N = 5
Disease: 0.95 +/- 0.045
Risk: 0.8 +/- 0.082
Mutation: 0.8 +/- 0.082
N = 10
Disease: 0.98 +/- 0.034
Risk: 0.85 +/- 0.11
Mutation: 0.89 +/- 0.082
N = 20
Disease: 0.98 +/- 0.034
Risk: 0.86 +/- 0.087
Mutation: 0.89 +/- 0.082
N = 100
Disease: 0.97 +/- 0.043
Risk: 0.76 +/- 0.086
Mutation: 0.88 +/- 0.11


ROC/PR
lednotlive datasety - contribution
framework zlepsit
MOGONET trial,
zalozit github